In [7]:
import numpy as np
import tensorflow
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import keras
from keras.datasets import mnist



(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("Training label shape: ", y_train.shape)
print("First 5 training labels: ", y_train[:5])

n_epochs = 3
batch_size_train = 64
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1

image_vector_size = 28*28
x_train = torch.from_numpy(x_train.reshape(x_train.shape[0], image_vector_size))/255
x_test = torch.from_numpy(x_test.reshape(x_test.shape[0], image_vector_size ))/255
y_train = y_train
y_test = y_test

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 10) 
        self.fc2 = nn.Linear(10, 1)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return F.softmax(x)
    
    def evaluate(self, x, y):
        proba = self.forward(x)
        y_pred = 1.0*(proba > 0.5)
        accuracy = 1.0*torch.sum(y_pred.reshape(-1,) == y)/proba.shape[0]
        return accuracy

network = Net()
criterion = torch.nn.BCELoss()
optimizer = optim.SGD(network.parameters(), lr=0.01)

epochs = 1

for e in range(epochs):
    network.train()
     
    optimizer.zero_grad()
    o = network.forward(x_train)
    loss = criterion(o, y_train)
    loss.backward()
    optimizer.step()

    if e%10 == 0:
        print(e, loss)

Training label shape:  (60000,)
First 5 training labels:  [5 0 4 1 9]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (60000x784 and 2x10)

In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time

n_epochs = 5
batch_size_train = 64
batch_size_test = 1000
learning_rate = 2.0
momentum = 0.5
log_interval = 100

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
    
def train(epoch):
      network.train()
      for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
          train_losses.append(loss.item())
          train_counter.append(
            (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
        
               
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

<ipython-input-8-23f681d135b7>:61: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
C:\Users\andre\miniconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3316, Accuracy: 1137/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.339086
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.326659
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.320415
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.327615
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.281480
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.308623
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.306325
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.327204
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.294496
Train Epoch: 1 [57600/60000 (96%)]	Loss: 2.344806

Test set: Avg. loss: 2.3070, Accuracy: 1135/10000 (11%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.292776
Train Epoch: 2 [6400/60000 (11%)]	Loss: 2.261912
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.325833
Train Epoch: 2 [19200/60000 (32%)]	Loss: 2.337439
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.322156
Train Epoch: 2 [32000/60000 (53%)]	Loss: 2.309683
Train Epoch: 2 [38400/60000 (64%)]	Loss: 2.298310
Train Epoch: 2 [44800/60000 (75%)]	Loss: 2.341